# Intro

{Fill in with information about this notebook}

# Set Up notebook 

In [2]:
#Import modules
import pandas as pd
import csv
import numpy as np
import pathlib
import os

#The rest of this is just for fun/quick dataviz at the end to map and ensure seems about right
import cartopy
import matplotlib.pyplot as plt
import rasterio
import geopandas

from lib import readData
from lib import mapping
from lib import cleanData
from lib import classify
from lib import exportData

todayDate, dateSuffix = readData.getCurrentDate()
repoDir = pathlib.Path(os.getcwd())

# Read in data

- Set up filepaths
- Read in data from:
    - downholeData table (from database)
    - headerData table (from database)
    - xyzData file (from previously carried out work) (will eventually make this updateable)

Read in data

In [3]:
directoryDir = r'\\isgs-sinkhole\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\\'[:-1]

downholeDataPATH, headerDataPATH, xyzInPATH  = readData.filesSetup(db_dir=directoryDir)

#Functions to read data into dataframes. Also excludes extraneous columns
headerDataIN, downholeDataIN = readData.readRawTxtData(downholefile=downholeDataPATH, headerfile=headerDataPATH)
xyzDataIN = readData.readXYZData(xyzfile=xyzInPATH)

Most Recent version of this file is : ISGS_DOWNHOLE_DATA_2023-01-06.txt
Most Recent version of this file is : ISGS_HEADER_2023-01-06.txt
Most Recent version of this file is : xyzData.csv
Using the following files:

\\isgs-sinkhole\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\ISGS_DOWNHOLE_DATA_2023-01-06.txt
\\isgs-sinkhole\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\ISGS_HEADER_2023-01-06.txt
\\isgs-sinkhole\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\xyzData.csv
Downhole Data has 3054409 valid well records.
Header Data has 636855 unique wells with valid location information.


Define datatypes (doing this during the read in process has presented issues)

In [4]:
#Define datatypes, to read into defineDataTypes() function
##EVENTUALLY, MAKE THIS A FILE IN THE RES FOLDER TO READ IN
#downholeDataDTYPES = {'ID':np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}
#headerDataDTYPES = {'ID':np.uint32,'API_NUMBER':np.uint64,"TDFORMATION":str,"PRODFORM":str,"TOTAL_DEPTH":np.float64,"SECTION":np.float64,"TWP":np.float64,"TDIR":str,"RNG":np.float64,"RDIR":str,"MERIDIAN":np.float64,"FARM_NAME":str,"NSFOOT":np.float64,"NSDIR":str,"EWFOOT":np.float64,"EWDIR":str,"QUARTERS":str,"ELEVATION":np.float64,"ELEVREF":str,"COMP_DATE":str,"STATUS":str,"FARM_NUM":str,"COUNTY_CODE":np.float64,"PERMIT_NUMBER":str,"COMPANY_NAME":str,"COMPANY_CODE":str,"PERMIT_DATE":str,"CORNER":str,"LATITUDE":np.float64,"LONGITUDE":np.float64,"ENTERED_BY":str,"UPDDATE":str,"ELEVSOURCE":str, "ELEV_FT":np.float64}
#xyzDataDTYPES = {'ID':np.uint64, 'API_NUMBER':np.uint64, "LATITUDE":np.float64, "LONGITUDE":np.float64, "ELEV_FT":np.float64}

#Define datatypes of each column of the new dataframes
downholeDataIN = readData.defineDataTypes(downholeDataIN, dtypeFile='downholeDataTypes.txt')
headerDataIN = readData.defineDataTypes(headerDataIN, dtypeFile='headerDataTypes.txt')
xyzDataIN = readData.defineDataTypes(xyzDataIN, dtypeFile='xyzDataTypes.txt')

#Make a copy of the data so raw data is preserved while we work with the rest of the data
downholeData = downholeDataIN.copy()
headerData = headerDataIN.copy()
xyzData = xyzDataIN.copy()

# Extract Elevation Data

Extract elevation data from consistent elevation dataset for all wells (lidar or other statewide DEM)

In [5]:
#First, get wells with updated xyz info
    #Check first if xyzData needs to be updated with locations (?)
    #Check which wells in headerData don't have associated lidar data

#statewideLidar =  ow
#mapping.rastertoPoints_extract()

Merge elevation data with headerData table

In [6]:
uniqueWells = headerData['API_NUMBER'].unique()
#xyzData['UniqueWells'] = uniqueWells

headerData = mapping.addElevtoHeader(xyzData, headerData)
##NEED TO UPDATE THIS TO WORK WITH DATA WITH NO XYZ ELEVATION DATA FROM LIDAR
#Change xyz column name to indicate lidar
#Use order of preference: lidar, headerData table?/30/10m DEM?

# Data Cleaning

## First, let's clean up records in the data without the necessary information

HeaderXYZ dataframe has headerdata with updated elevations (from lidar or other DEM)

Now, remove data from downholeData table that does not have location information (Since we would not know where to put it anyway)

In [7]:
downholeData = cleanData.removeNonlocatedData(downholeData, headerData)

13615 records removed without location information.


Clip data from outside Illinois

In [8]:
#Not done yet, use geopandas clip (May need to generate geometry column first in headerData)

In [9]:
headerData

,API_NUMBER,TOTAL_DEPTH,SECTION,TWP,TDIR,RNG,RDIR,MERIDIAN,QUARTERS,ELEVATION,ELEVREF,COUNTY_CODE,ELEVSOURCE,LATITUDE,LONGITUDE,ELEV_FT
0,4045883208,570.0,12.0,1.0,N,5.0,W,4.0,SW SE,620.0,GL,1.0,nan,40.080723,-90.921097,616.728271
1,4045883308,20.0,12.0,1.0,N,5.0,W,4.0,SE SE,590.0,GL,1.0,nan,40.079857,-90.913589,580.442200
2,4045883408,725.0,20.0,1.0,N,5.0,W,4.0,SE,731.0,TM,1.0,nan,40.051773,-90.995911,679.606506
3,4045883508,1129.0,22.0,1.0,N,5.0,W,4.0,SE NE NW,732.0,GL,1.0,nan,40.059509,-90.963264,737.663574
4,4045883608,732.0,24.0,1.0,N,5.0,W,4.0,SW,745.0,TM,1.0,nan,40.054066,-90.932716,741.997925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636850,3848631244,139.0,30.0,1.0,N,21.0,E,0.0,SW NE NW,817.0,nan,59.0,GPS,42.522205,-88.064186,NaN
636851,3848631344,50.0,34.0,1.0,N,19.0,E,0.0,SW SW SE,975.0,nan,59.0,nan,42.495945,-88.232162,0.000000
636852,3848657344,209.0,21.0,1.0,N,20.0,E,0.0,SE SE,780.0,nan,59.0,GPS,42.526997,-88.128387,NaN
636853,3848658244,200.0,36.0,1.0,N,20.0,E,0.0,SE NE,840.0,nan,59.0,TM,42.504860,-88.075172,0.000000


Remove headerData rows without surface elevation information (this currently clips data from outside Illinois)

In [10]:
headerData = cleanData.removenotopo(headerData, printouts=True)

Number of rows before dropping those without surface elevation information: 636855
Number of rows after dropping those without surface elevation information: 632702
Well records deleted: 4153


Remove rows from downholeData with no depth information and where depth information is obviously bad (i.e., top depth > bottom depth)

In [11]:
#Drop records with no depth information
donwholeData = cleanData.dropnodepth(downholeData, printouts=True)
#Drop records with bad depth information (i.e., top depth > bottom depth) (Also calculates thickness of each record)
donwholeData = cleanData.dropbaddepth(downholeData, printouts=True)

Number of rows before dropping those without record depth information: 3040794
Number of rows after dropping those without record depth information: 2625120
Number of well records without formation information deleted: 415674
Number of rows before dropping those with obviously bad depth information: 3040794
Number of rows after dropping those with obviously bad depth information: 2622533
Well records deleted: 418261


Drop records with no FORMATION information

In [12]:
downholeData = cleanData.dropnoformation(downholeData, printouts=True)

Number of rows before dropping those without FORMATION information: 3040794
Number of rows after dropping those without FORMATION information: 3040794
Well records deleted: 0


Now we are going to export this data, to have record of cleaned data

In [14]:
downholeData.reset_index(inplace=True,drop=True)
headerData.reset_index(inplace=True,drop=True)

downholeData.to_csv(str(repoDir)+'/out/downholeData_cleaned'+dateSuffix+'.csv',index_label='ID')
headerData.to_csv(str(repoDir)+'/out/headerData_cleaned'+dateSuffix+'.csv',index_label='ID')

# Classification

The following flags are used to mark the classification method:
- 0: Not classified
- 1: Specific Search Term Match
- 2: Wildcard match (startTerm) - no context
- 3: Bedrock classification for obvious bedrock
- 4: Wildcard match (startTerm) - with context

In [16]:
#Read in dictionary files for downhole data
specTermsPATH, startTermsPATH = readData.searchTermFilePaths()

Most Recent version of this file is : SearchTerms-Specific_2022-11-16_essCols.csv
Most Recent version of this file is : SearchTerms-Start.csv


In [18]:
specTerms, startTerms = readData.readSearchTerms(specfile=specTermsPATH, startfile=startTermsPATH)

Join the dataframes--for the specific search terms, this is the same as classifying them

In [19]:
downholeData_spec = classify.specificDefine(downholeData, specTerms, printouts=True)
downholeData = downholeData_spec.copy()

Records Classified with full search term: 2138359
Records Classified with full search term: 70.32% of data


Create a dataframe with only the records already classified (using the specific search terms in this case, classifiedDF), and one that still needs to be searched (searchDF)

In [20]:
classifedDF, searchDF = classify.splitDefined(downholeData)
searchDF.shape[0]

902435

Now, do the classification routine on the searchDF database

In [21]:
searchDF = classify.startDefine(df=searchDF, starterms=startTerms, printouts=True)

Start Term process should be done by 14:54


c:\Users\balikian\OneDrive - University of Illinois - Urbana\CodesandScripts\Git\wells4hydrogeology\lib\classify.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLASS_FLAG'].where(~df['FORMATION'].str.startswith(s,na=False),4,inplace=True)
c:\Users\balikian\OneDrive - University of Illinois - Urbana\CodesandScripts\Git\wells4hydrogeology\lib\classify.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['INTERPRETATION'].where(~df['FORMATION'].str.startswith(s,na=False),starterms.loc[i,'INTERPRETATION'],inplace=True)


Records classified with start search term: 107010
Records classified with start search term: 11.86% of remaining data


c:\Users\balikian\OneDrive - University of Illinois - Urbana\CodesandScripts\Git\wells4hydrogeology\lib\classify.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BEDROCK_FLAG'] = df["INTERPRETATION"] == 'BEDROCK'


Merge specDF and searchDF back together all back in single dataframe

In [23]:
downholeData_Terms = classify.remergeData(classifieddf=classifedDF, searchdf=searchDF)
downholeData = downholeData_Terms.copy()

Export terms that still need to be defined to csv (along with their counts)

In [24]:
classify.export_toBeDefined(downholeData, str(repoDir)+'/out/')

Classify all  data under depth threshold (default is 550') as bedrock (should not be an issue, but just in case)

In [25]:
classifedDF, searchDF = classify.splitDefined(downholeData)
searchDF = classify.depthDefine(searchDF, thresh=550, printouts=True)
downholeData_Class = classify.remergeData(classifieddf=classifedDF, searchdf=searchDF)
downholeData = downholeData_Class.copy()

c:\Users\balikian\OneDrive - University of Illinois - Urbana\CodesandScripts\Git\wells4hydrogeology\lib\classify.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLASS_FLAG'].mask(df['TOP']>thresh, 3 ,inplace=True) #Add a Classification Flag of 3 (bedrock b/c it's deepter than 550') to all records where the top of the interval is >550'
c:\Users\balikian\OneDrive - University of Illinois - Urbana\CodesandScripts\Git\wells4hydrogeology\lib\classify.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BEDROCK_FLAG'].mask(df['TOP']>thresh, True, inplace=True)


Records classified deeper than 550' :79219
Records classified deeper than 550', as a percentage of remaining unclassified records: 11.06%
Total bedrock records classified: 158438
This is now a total of 19.92% of the data classified.


Add '0' flag for data still not classified

In [26]:
downholeData = classify.fillUnclassified(downholeData)

In [27]:
downholeData['CLASS_FLAG'].value_counts()

1.0    2138359
0.0     716206
4.0     107010
3.0      79219
Name: CLASS_FLAG, dtype: int64

## Add "Flag" for target interpratations

In [28]:
#dictDir = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\SupportingDocs\\"
targetInterpDF = readData.readLithologies()

AttributeError: module 'lib.readData' has no attribute 'readLithologies'

In [ ]:
downholeData = classify.mergeLithologies(downholedata=downholeData, targinterps=targetInterpDF)

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,TARGET
0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,0.0,-2
1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,0.0,-2
2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,0.0,-2
3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,0.0,-2
4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,0.0,-2
...,...,...,...,...,...,...,...,...,...
2588272,480590043000,WFORMATIONS,"red clay, firm",120.0,45.0,165.0,NaN,0.0,-2
2588273,480590043000,WFORMATIONS,sand silt & gravel,165.0,6.0,171.0,NaN,0.0,-2
2588274,480590043000,WFORMATIONS,"sandy blue clay, soft",85.0,35.0,120.0,NaN,0.0,-2
2588275,480590043000,WFORMATIONS,stoney blue clay,18.0,67.0,85.0,NaN,0.0,-2


Flags used for target classification purposes:
- -2: No classification 
- -1: Classified, not used/not definitive
- 0: Classified, not target material
- 1: Classified as target material

In [ ]:
downholeData['TARGET'].value_counts()

-2    1337019
-1     722307
0      336977
1      191974
Name: TARGET, dtype: int64

Find all unique wells in downhole dataset

In [ ]:
#Get Unique well APIs
wellsDF = classify.getUniqueWells(downholeData)

Sort dataset by API Number and Depth of top of record (will be easier to do data analysis with records in the correct order)

In [ ]:
downholeData_sorted = downholeData.sort_values(['API_NUMBER','TOP'])
downholeData_sorted.reset_index(inplace=True)
downholeData_sorted

,index,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,TARGET
0,2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,0.0,-2
1,0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,0.0,-2
2,1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,0.0,-2
3,4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,0.0,-2
4,3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,0.0,-2
...,...,...,...,...,...,...,...,...,...,...
2588272,2588273,480590043000,WFORMATIONS,sand silt & gravel,165.0,6.0,171.0,NaN,0.0,-2
2588273,2588276,480590043000,WFORMATIONS,yellow clay,0.0,18.0,18.0,CLAY,1.0,0
2588274,2588274,480590043000,WFORMATIONS,"sandy blue clay, soft",85.0,35.0,120.0,NaN,0.0,-2
2588275,2588272,480590043000,WFORMATIONS,"red clay, firm",120.0,45.0,165.0,NaN,0.0,-2


# Get Bedrock Depth and Layer Thickness

Read in Study Area

In [ ]:
studyAreaPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ESL_StudyArea_5mi.shp"
studyAreaIN, saExtent = mapping.readStudyArea(studyAreaPath)

Read in/Define Model Grid

In [ ]:
modelGridPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\grid_625_raster.tif"
modelGrid = mapping.readModelGrid(saExtent=saExtent, gridpath=modelGridPath, readGrid=True)

Read in surface elevation grid

In [ ]:
surfaceElevPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ILStateLidar_ClipExtentESL.tif"
surfaceElevGridIN = mapping.readSurfaceGrid(surfaceelevpath=surfaceElevPath)

Read in bedrock elevation grid

In [ ]:
bedrockElevPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ESLBedrock.tif"
bedrockElevGridIN=mapping.readBedrockGrid(bedrockelevpath=bedrockElevPath)

Plot just to see them

In [ ]:
fig, ax = plt.subplots(ncols = 2, nrows=1)
bedrockElevGridIN.plot(ax=ax[0])
surfaceElevGridIN.plot(ax=ax[1])

Reproject and align raster grids for surface elevation and bedrock topo (reproject well data too if needed)

In [ ]:
bedrockGrid, surfaceGrid = mapping.alignRasters(bedrockgrid=bedrockElevGridIN, surfacegrid=surfaceElevGridIN, modelgrid=modelGrid)

fig, ax = plt.subplots(ncols = 2, nrows=1)
bedrockGrid.plot(ax=ax[0])
surfaceGrid.plot(ax=ax[1])

Use the surface elevation raster and bedrock elevation raster to get depth to bedrock

In [ ]:
driftThickGrid, layerThickGrid = mapping.getDriftThick(surface=surfaceGrid, bedrock=bedrockGrid, noLayers=9, plotData=True)

# Work here next!

Now, sample each well point (headerData) to get layer thickness, surface elevation, and bedrock 

In [ ]:
#TO DO
#PRevious code that might be useful
#xyzData = pd.read_csv(processDirectory+"headerData__essCols_elev10mDTM.csv",usecols = ['API_NUMBER', 'LONGITUDE','LATITUDE', '10mDTM_ft'])
#downhole_XYZ = pd.merge(downholeData_sorted, xyzData.set_index('API_NUMBER'), on='API_NUMBER',how='left')
#downhole_XYZ['TOP_ELEV_FT'] = downhole_XYZ['10mDTM_ft'] - downhole_XYZ['TOP']
#downhole_XYZ['BOT_ELEV_FT'] = downhole_XYZ['10mDTM_ft'] - downhole_XYZ['BOTTOM']
#downhole_XYZ['TOP_ELEV_M'] = downhole_XYZ['TOP_ELEV_FT'] * 0.3048
#downhole_XYZ['BOT_ELEV_M'] = downhole_XYZ['BOT_ELEV_FT'] * 0.3048
#downhole_XYZ

Calculate  all layer depths/elevations at all wells

In [ ]:
#TO DO

Define function to calculate target thickness in each layer

In [ ]:
##THIS CELL MAY NEED TO BE UPDATED!!!!!


#Define the function to export the result of thickness of target sediments in each layer
def Layers_surfacedown(df, layer = 1):
    
    #Generate Column names based on (looped) integers
    topCol = "ESL_ModelTopoLyrs_"+str(layer)
    if layer != 9: #For all layers except the bottom layer....
        botCol = "ESL_ModelTopoLyrs_"+str(layer+1) #use the layer below it to 
    else: #Otherwise, ...
        botCol = "BedrockCorr" #Use the (corrected) bedrock depth

    #Divide records into 4 separate categories for ease of calculation, to be joined back together later  
        #Category 1: Well interval starts above layer top, ends within model layer
        #Category 2: Well interval is entirely contained withing model layer
        #Category 3: Well interval starts within model layer, continues through bottom of model layer
        #Category 4: well interval begins and ends on either side of model layer (model layer is contained within well layer)

    #records1 = intervals that go through the top of the layer and bottom is within layer
    records1 = df.loc[(df['TOP_ELEV_ft'] > df[topCol]) & (df['BOT_ELEV_ft'] > df[botCol]) & (df['BOT_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records1['TARG_THICK'] = pd.DataFrame(np.round((records1.loc[:,topCol]-records1.loc[: , 'BOT_ELEV_ft']) * records1['Target'],3)).copy()
    
    #records2 = entire interval is within layer
    records2 = df.loc[(df['TOP_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] >= df[botCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records2['TARG_THICK'] = pd.DataFrame(np.round((records2.loc[: , 'TOP_ELEV_ft'] - records2.loc[: , 'BOT_ELEV_ft']) * records2['Target'],3)).copy()

    #records3 = intervals with top within layer and bottom of interval going through bottom of layer
    records3 = df.loc[(df['TOP_ELEV_ft'] > df[botCol]) & (df['BOT_ELEV_ft'] < df[botCol]) & (df['TOP_ELEV_ft'] <= df[topCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records3['TARG_THICK'] = pd.DataFrame(np.round((records3.loc[: , 'TOP_ELEV_ft'] - (records3.loc[:,botCol]))*records3['Target'],3)).copy()

    #records4 = interval goes through entire layer
    records4 = df.loc[(df['TOP_ELEV_ft'] > df[topCol]) & (df['BOT_ELEV_ft'] < df[botCol]) & (df['BOT_ELEV_ft'] <= df['TOP_ELEV_ft'])].copy()
    records4['TARG_THICK'] = pd.DataFrame(np.round((records4.loc[: , topCol]-records4.loc[: , botCol]) * records4['Target'],3)).copy()

    
    #Put the four calculated record categories back together into single dataframe
    res = records1.append(records2).append(records3).append(records4)
    
    res_df = res.groupby(['API_NUMBER','LATITUDE','LONGITUDE'],as_index=False).sum()#calculate thickness for each well interval in the layer indicated (e.g., if there are two well intervals from same well in one model layer)

    res_df['TARG_THICK_PER'] = pd.DataFrame(np.round(res_df['TARG_THICK']/res_df['LyrThick'],3)) #Calculate thickness as percent of total layer thickness
    res_df["LAYER"] = layer #Just to have as part of the output file, include the present layer in the file itself as a separate column
    res_df = res_df[['API_NUMBER', 'LATITUDE', 'LONGITUDE', 'TOP', 'BOTTOM','SURF_ELEV_ft', 'TOP_ELEV_ft', 'BOT_ELEV_ft',topCol,botCol,'LyrThick','TARG_THICK', 'TARG_THICK_PER', 'LAYER']].copy() #Format dataframe for output
    
    return res, res_df

Now run that function over all the layers, looping through each one

In [ ]:
#THIS CELL WILL NEED TO BE UPDATED

outDIR = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\MetroEast_HydroGeo\\CodeOutput\\"+codeTarget+"\\"

for i in np.arange(1,10):
    res, res_df = Layers_surfacedown(df, layer = i)#Run the function defined above for each layer
    outputname = codeTargShort+'Lyr'+str(i)+'.csv' #Create a filename based on the layer and target
    res_df.to_csv(outDIR+outputname)  #Export the file to csv
    #Could also potentially save these to variables for use in following cells

NameError: name 'codeTarget' is not defined

# Interpolate thickness values in each layer

Loop through each layer and interpolate (use same parameters (?))

Ensure rasters align (are co-registered) with grid

# Export

In [ ]:
#Export data 
downhole_bedrockDepth_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\Downhole_BedrockPicks.csv',index_label="ID")
wPermits_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\wPermits_BedrockPicks.csv',index_label="ID")